In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import pickle
import sys, os, time

sys.path.append(os.path.abspath("../src"))
from slp_model_setup import SegLabel, SLP_Model

In [2]:
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library.n_local.qaoa_ansatz import QAOAAnsatz
from qiskit.algorithms import VQE

from qiskit import Aer, transpile
from qiskit.utils import algorithm_globals, QuantumInstance

from qiskit.providers.fake_provider import FakeGuadalupeV2
from qiskit.providers.fake_provider import FakeJakartaV2
from qiskit.providers.fake_provider import FakeMumbaiV2
from qiskit.providers.fake_provider import FakeWashingtonV2

In [3]:
def get_resources_info(ansatz, backend, p=1, seed=12345):
    """ """

    n_runs = 10

    np.random.seed(seed)
    seeds = np.random.randint(0, 2**16, size=(n_runs,))

    resources = []
    for _seed in seeds:
        algorithm_globals.random_seed = _seed
        _ansatz = transpile(
            ansatz,
            backend,
            seed_transpiler=_seed,
        )
        resources.append(
            {
                "p": p,
                "num_qubits": _ansatz.num_qubits,
                "depth": _ansatz.depth(),
                "gates": list(_ansatz.count_ops().items()),
            }
        )
    return resources

In [4]:
def count_avg_resources(resources):
    """ """
    n_qubits = resources[0]["num_qubits"]
    depth = []

    gates = dict()
    for k in dict(resources[0]["gates"]).keys():
        gates[k] = []

    for entry in resources:
        depth.append(entry["depth"])
        d = dict(entry["gates"])
        for k, v in d.items():
            gates[k].append(v)

    print("\nResources:")
    print("p:", resources[0]["p"])
    print("n_qubits:", resources[0]["num_qubits"])
    print("depth:", np.mean(depth), np.std(depth))
    for k, v in gates.items():
        print(k, np.mean(v), np.std(v))

## Model

In [5]:
with open("../data/hip-dataset.pickle", "rb") as f:
    segLabel = pickle.load(f)

num_nodes = segLabel.num_detections
num_labels = segLabel.num_classes
num_segments = 2  # segLabel.max_num_persons

# alpha_dc
A = segLabel.alphas.copy()
A[A > 100] = 100
# print(A)

# beta_dc_d'c'
B = segLabel.betas.copy()
# B = np.nan_to_num(B, nan=0.0)
# print(B)

model = SLP_Model(num_nodes, num_segments, num_labels, A, B)

In [6]:
# Build the DOcplex model of the S&L problem
mdl = model.build_model(D=100)
mdl.solve()
mdl.print_information()
mdl.print_solution()

x = np.zeros(mdl.number_of_binary_variables)
for v in mdl.iter_binary_vars():
    x[v.index] = v.solution_value
# print(x)

obj_exact = mdl.objective_value
x_exact = x

Model: Segmentation and Labeling
 - number of variables: 16
   - binary=16, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: minimize quadratic
 - problem type is: MIQP
objective: -2.200
status: OPTIMAL_SOLUTION(2)
  x_0_0_0=1
  x_1_0_1=1
  x_2_1_0=1
  x_3_1_1=1


## QAOA

In [7]:
mdl_qp = from_docplex_mp(mdl)
# print(mdl_qp.prettyprint())

mdl_qubo = QuadraticProgramToQubo().convert(mdl_qp)

qubitOp, offset = mdl_qubo.to_ising()

if True:
    # normalize the Hamiltonian
    w_max = np.max(np.abs(qubitOp.primitive.coeffs))
    qubitOp.primitive.coeffs /= w_max
    offset /= w_max

print("Offset:", offset)
# print("Ising Hamiltonian:\n", str(qubitOp))

Offset: 7.760113076590407


In [8]:
# QUBO matrix info
q2 = mdl_qubo.objective.quadratic.to_dict()
num_qubits = qubitOp.num_qubits
num_q2 = len(q2) - num_qubits
print("Number of quadratic terms: ", num_q2)
print("QUBO matrix sparsity: ", num_q2 / (num_qubits * (num_qubits - 1) * 0.5))

Number of quadratic terms:  72
QUBO matrix sparsity:  0.6


## Standard QAOA ansatz

In [9]:
num_qubits = qubitOp.num_qubits
reps = 1

# QAOA ansatz
qc = QAOAAnsatz(qubitOp)

## Rearranged QAOA ansatz

In [10]:
def find(s, ch):
    return tuple(i for i, ltr in enumerate(s) if ltr == ch)


op_list = qubitOp.primitive.to_list()

H_dict = dict()
for op in op_list:
    k = find(op[0], "Z")
    H_dict[k] = op[1]

ops = []
for p in range(reps):
    permutations = np.arange(num_qubits)  # To decrease the depth of the circuit
    for qbits, value in H_dict.items():
        s = ["I"] * num_qubits
        if len(qbits) == 1:
            s[qbits[0]] = "Z"
            ops.append(("".join(s), value))
    for jj in range(num_qubits):
        starting_qubit = jj % 2
        for k in range(starting_qubit, num_qubits - 1, 2):
            qubit_pair = (permutations[k], permutations[k + 1])
            if qubit_pair in H_dict.keys():
                s = ["I"] * num_qubits
                s[qubit_pair[0]] = "Z"
                s[qubit_pair[1]] = "Z"
                ops.append(("".join(s), H_dict[qubit_pair]))
            elif qubit_pair[::-1] in H_dict.keys():
                qubit_pair = qubit_pair[::-1]
                s[qubit_pair[0]] = "Z"
                s[qubit_pair[1]] = "Z"
                ops.append(("".join(s), H_dict[qubit_pair]))
            permutations[[k, k + 1]] = permutations[[k + 1, k]]

qubitOp_1 = qubitOp.from_list(ops)
qc_1 = QAOAAnsatz(qubitOp_1, reps=reps)

## QASM

In [11]:
p = 1

backend = Aer.get_backend("qasm_simulator")

print("Unorganized (original) circuit:")
resources = get_resources_info(qc, backend, p=p)
count_avg_resources(resources)

Unorganized (original) circuit:

Resources:
p: 1
n_qubits: 16
depth: 26.0 0.0
rzz 72.0 0.0
h 16.0 0.0
rz 16.0 0.0
rx 16.0 0.0


In [12]:
p = 1

backend = Aer.get_backend("qasm_simulator")

print("Rearranged circuit:")
resources = get_resources_info(qc_1, backend, p=p)
count_avg_resources(resources)

Rearranged circuit:

Resources:
p: 1
n_qubits: 16
depth: 19.0 0.0
rzz 72.0 0.0
h 16.0 0.0
rz 16.0 0.0
rx 16.0 0.0


## Guadalupe

In [13]:
p = 1

backend = FakeGuadalupeV2()

print("Unorganized (original) circuit:")
resources = get_resources_info(qc, backend, p=p)
count_avg_resources(resources)

Unorganized (original) circuit:

Resources:
p: 1
n_qubits: 16
depth: 182.3 14.765161699080712
cx 297.3 17.849649856509792
rz 168.0 0.0
sx 48.0 0.0


In [14]:
p = 1

backend = FakeGuadalupeV2()

print("Rearranged circuit:")
resources = get_resources_info(qc_1, backend, p=p)
count_avg_resources(resources)

Rearranged circuit:

Resources:
p: 1
n_qubits: 16
depth: 185.7 15.659182609574485
cx 365.2 15.606408939919524
rz 168.0 0.0
sx 48.0 0.0


## Mumbai

In [15]:
p = 1

backend = FakeMumbaiV2()

print("Unorganized (original) circuit:")
resources = get_resources_info(qc, backend, p=p)
count_avg_resources(resources)

Unorganized (original) circuit:

Resources:
p: 1
n_qubits: 27
depth: 217.3 36.0528778324283
cx 343.6 39.0517605236947
rz 168.0 0.0
sx 48.0 0.0


In [16]:
p = 1

backend = FakeMumbaiV2()

print("Rearranged circuit:")
resources = get_resources_info(qc_1, backend, p=p)
count_avg_resources(resources)

Rearranged circuit:

Resources:
p: 1
n_qubits: 27
depth: 194.4 25.4566297847928
cx 393.5 37.617150343958805
rz 168.0 0.0
sx 48.0 0.0


## Washington

In [17]:
p = 1

backend = FakeWashingtonV2()

print("Unorganized (original) circuit:")
resources = get_resources_info(qc, backend, p=p)
count_avg_resources(resources)

Unorganized (original) circuit:

Resources:
p: 1
n_qubits: 127
depth: 208.8 27.24628415032039
cx 345.5 34.78289809662214
rz 168.0 0.0
sx 48.0 0.0


In [18]:
p = 1

backend = FakeWashingtonV2()

print("Rearranged circuit:")
resources = get_resources_info(qc_1, backend, p=p)
count_avg_resources(resources)

Rearranged circuit:

Resources:
p: 1
n_qubits: 127
depth: 203.3 30.60408469469394
cx 398.3 38.071117661555455
rz 168.0 0.0
sx 48.0 0.0
